In [3]:
import numpy as np
import pandas as pd
import os

os.chdir(r'C:\Users\user\Desktop\add')

In [79]:


class CUSUMDetector:
    def __init__(self, phase1_len=50, threshold=5.0, drift=0.5):
        """
        phase1_len: Phase I(기준) 구간 길이 (샘플 수)
        threshold: decision.interval (σ 단위)
        drift: se.shift (σ 단위)
        """
        self.phase1_len = phase1_len
        self.threshold = threshold
        self.drift = drift
        self.reset()

    def reset(self):
        """내부 상태 초기화"""
        self.mean_ = None
        self.sigma_ = None
        self.k = None
        self.h = None
        self.S_pos = 0.0
        self.S_neg = 0.0

    def fit(self, X: np.ndarray):
        """
        Phase I 데이터로부터 μ₀, σ₀, k, h 계산
        X: 1D array-like (phase1_len 길이)
        """
        x1 = np.asarray(X, dtype=float)
        if x1.ndim != 1 or len(x1) < self.phase1_len:
            raise ValueError("fit()에는 길이 phase1_len 이상의 1D 배열을 넣어야 합니다.")
        # 기준 구간
        p1 = x1[:self.phase1_len]
        self.mean_  = p1.mean() # axis = 0 추가해서 평균벡터로 만들기
        self.sigma_ = p1.std(ddof=1) # axis = 0 추가해서 공분산 행렬로 만들기
        # qcc::cusum 매핑
        # k = se.shift * σ₀ / 2
        self.k = (self.drift * self.sigma_) / 2.0
        # h = decision.interval * σ₀
        self.h = self.threshold * self.sigma_
        # 누적합 초기화
        self.S_pos = 0.0
        self.S_neg = 0.0

    def update(self, x: np.ndarray) -> bool:
        """
        Phase II 한 점씩 호출
        x: 단일 관측값 (스칼라)
        Returns:
          True  → S_pos > h 이거나 –S_neg > h 이면 (상향/하향 탐지)
          False → 아직 탐지 아님
        """
        if self.mean_ is None:
            raise RuntimeError("먼저 fit()을 호출하여 Phase I를 설정하세요.")
        xi = float(x)
        s = xi - self.mean_ - self.k
        self.S_pos = max(0.0, self.S_pos + s)
        self.S_neg = min(0.0, self.S_neg + s)
        # 상향 또는 하향 임계 초과 여부
        return (self.S_pos > self.h) or ((-self.S_neg) > self.h)


In [5]:
def load_data_long(file_path: str):
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()
    pivot = df.pivot_table(
        index=["time", "Label"],
        columns="Variable",
        values="Value",
        aggfunc="first"
    ).sort_index()
    X_raw = pivot.to_numpy(dtype=float)
    y_raw = np.array([lab for (_, lab) in pivot.index], dtype=int)
    return X_raw, y_raw, {c: c for c in np.unique(y_raw)}

In [7]:
def create_training_windows(X: np.ndarray, y: np.ndarray, window_size: int, step: int = 1):
    if X.ndim == 2 and X.shape[0] < X.shape[1]:
        X = X.T
    T, F = X.shape
    windows, labels = [], []
    for start in range(0, T - window_size + 1, step):
        end = start + window_size
        win = X[start:end]
        windows.append(win)
        labels.append(y[end - 1])
    X_windows = np.stack(windows)
    y_windows = np.array(labels)
    return X_windows, y_windows

In [37]:
df = pd.read_csv('Et_H_CO.csv')
df.columns = df.columns.str.strip()
pivot = df.pivot_table(
    index=["time", "Label"],
    columns="Variable",
    values="Value",
    aggfunc="first"
).sort_index()
pivot.to_numpy(dtype = float)

array([[629., 719., 331., ..., 572., 566., 700.],
       [644., 737., 334., ..., 548., 584., 727.],
       [665., 759., 342., ..., 598., 593., 739.],
       ...,
       [721., 815., 359., ..., 705., 675., 853.],
       [730., 829., 359., ..., 656., 686., 865.],
       [742., 843., 363., ..., 707., 694., 872.]])

In [6]:
X_raw, y_raw, label_mapping = load_data_long("Et_H_CO.csv")

In [8]:
window_size = 5
X_train, y_train = create_training_windows(X_raw, y_raw, window_size)

In [70]:
X_raw[:10,]

array([[629., 719., 331., 273., 518., 572., 566., 700.],
       [644., 737., 334., 272., 521., 548., 584., 727.],
       [665., 759., 342., 275., 531., 598., 593., 739.],
       [660., 754., 341., 279., 533., 627., 592., 735.],
       [630., 719., 331., 274., 518., 571., 566., 700.],
       [643., 736., 333., 272., 521., 548., 583., 727.],
       [665., 759., 342., 275., 531., 598., 593., 739.],
       [662., 754., 341., 279., 533., 626., 592., 735.],
       [630., 719., 333., 273., 518., 571., 566., 701.],
       [643., 736., 334., 272., 519., 547., 584., 728.]])

In [68]:
X_train

array([[[629., 719., 331., ..., 572., 566., 700.],
        [644., 737., 334., ..., 548., 584., 727.],
        [665., 759., 342., ..., 598., 593., 739.],
        [660., 754., 341., ..., 627., 592., 735.],
        [630., 719., 331., ..., 571., 566., 700.]],

       [[644., 737., 334., ..., 548., 584., 727.],
        [665., 759., 342., ..., 598., 593., 739.],
        [660., 754., 341., ..., 627., 592., 735.],
        [630., 719., 331., ..., 571., 566., 700.],
        [643., 736., 333., ..., 548., 583., 727.]],

       [[665., 759., 342., ..., 598., 593., 739.],
        [660., 754., 341., ..., 627., 592., 735.],
        [630., 719., 331., ..., 571., 566., 700.],
        [643., 736., 333., ..., 548., 583., 727.],
        [665., 759., 342., ..., 598., 593., 739.]],

       ...,

       [[721., 816., 359., ..., 706., 676., 852.],
        [732., 830., 358., ..., 657., 684., 864.],
        [741., 843., 362., ..., 707., 695., 873.],
        [721., 821., 357., ..., 597., 681., 856.],
        [721

In [80]:
cusum = CUSUMDetector(phase1_len=20, threshold=5.0, drift=0.0)
cusum

In [81]:
cusum.fit(X_raw[:20].mean(axis=1))

In [86]:
print(cusum.mean_)
print(cusum.sigma_)
print(cusum.k)
print(cusum.h)


552.99375
11.536802881374207
0.0
57.684014406871036


In [42]:
x1 = np.asarray(X_raw[:20], dtype=float)
x1.ndim

2

In [77]:
X_raw.shape


(11880, 8)

In [ ]:
p1 = x1[:20]
print(p1.mean()) # axis = 0이 되어야 각 센서에 대한 평균이 나올텐데
print(p1.std(ddof=1)) # axis = 0이 되어야 각 센서에 대한 표준편차가 나올텐데 ?

552.99375
[14.55877381 16.2683904   4.7047009   2.76443579  6.73150488 30.08514234
 11.08092244 15.41453518]


In [54]:
def create_expanding_windows(X: np.ndarray, y: np.ndarray, min_size: int = 1):
    if X.ndim == 2 and X.shape[0] < X.shape[1]:
        X = X.T
    T, F = X.shape
    windows_list = []
    labels = []
    for end in range(min_size, T + 1):
        win = X[:end]
        windows_list.append(win)
        labels.append(y[end - 1])
    y_windows = np.array(labels)
    return windows_list, y_windows

In [55]:
X_exp, _ = create_expanding_windows(X_raw, y_raw, min_size=1)

In [65]:
X_raw[:10,]

array([[629., 719., 331., 273., 518., 572., 566., 700.],
       [644., 737., 334., 272., 521., 548., 584., 727.],
       [665., 759., 342., 275., 531., 598., 593., 739.],
       [660., 754., 341., 279., 533., 627., 592., 735.],
       [630., 719., 331., 274., 518., 571., 566., 700.],
       [643., 736., 333., 272., 521., 548., 583., 727.],
       [665., 759., 342., 275., 531., 598., 593., 739.],
       [662., 754., 341., 279., 533., 626., 592., 735.],
       [630., 719., 333., 273., 518., 571., 566., 701.],
       [643., 736., 334., 272., 519., 547., 584., 728.]])

In [62]:
X_exp

[array([[629., 719., 331., 273., 518., 572., 566., 700.]]),
 array([[629., 719., 331., 273., 518., 572., 566., 700.],
        [644., 737., 334., 272., 521., 548., 584., 727.]]),
 array([[629., 719., 331., 273., 518., 572., 566., 700.],
        [644., 737., 334., 272., 521., 548., 584., 727.],
        [665., 759., 342., 275., 531., 598., 593., 739.]]),
 array([[629., 719., 331., 273., 518., 572., 566., 700.],
        [644., 737., 334., 272., 521., 548., 584., 727.],
        [665., 759., 342., 275., 531., 598., 593., 739.],
        [660., 754., 341., 279., 533., 627., 592., 735.]]),
 array([[629., 719., 331., 273., 518., 572., 566., 700.],
        [644., 737., 334., 272., 521., 548., 584., 727.],
        [665., 759., 342., 275., 531., 598., 593., 739.],
        [660., 754., 341., 279., 533., 627., 592., 735.],
        [630., 719., 331., 274., 518., 571., 566., 700.]]),
 array([[629., 719., 331., 273., 518., 572., 566., 700.],
        [644., 737., 334., 272., 521., 548., 584., 727.],
    